# PyCity Schools Analysis

Generating merged DataFrame from both csv files

In [1]:
#import dependencies
import numpy as np
import pandas as pd

In [2]:
#import/read csv files
schools = "Resources/schools_complete.csv"
schoolsdf = pd.read_csv(schools) #, encoding = "utf-8", low_memory = False)

students = "Resources/students_complete.csv"
studentsdf = pd.read_csv(students) #, encoding = "utf-8", low_memory = False)

In [3]:
#Renaming columns (to lowercase and new names)

schoolsdf = schoolsdf.rename(index = str, columns = {"School ID":"school_id",
                                                     "name":"school_name",
                                                     "type":"school_type",
                                                     "size":"school_size",
                                                     "budget":"school_budget"})

studentsdf = studentsdf.rename(index = str, columns = {"Student ID":"student_id",
                                                       "name":"student_name",
                                                       "gender":"student_gender",
                                                       "grade":"student_grade",
                                                       "school":"school_name"
                                                       })

#Adding cols to Student Data in order to calculate percentages
studentsdf['student_tracker'] = 1
studentsdf['reading_pass'] = np.where(studentsdf['reading_score'] >= 70, 1, 0)
studentsdf['math_pass'] = np.where(studentsdf['math_score'] >= 70, 1, 0)

#Merge DataFrames on 'school_name'
mergedf = pd.merge(schoolsdf, studentsdf, on = 'school_name'); #mergedf


In [4]:
#import pandas as pd
#from pandas import ExcelWriter
#from pandas import ExcelFile
#import numpy as np
 
#writer = ExcelWriter('Schools.xlsx')
#mergedf.to_excel(writer,'Sheet1',index=False)
#writer.save()

District Summary

In [15]:
#District Summary: Calculate key stats and create a new df summary
Total_Schools = schoolsdf["school_name"].count()
Total_Budget = schoolsdf["school_budget"].sum()
school_sub = [Total_Schools, Total_Budget]

#studentsdf.head(5)

Total_Students = studentsdf['student_tracker'].sum()
Math_Avg = studentsdf['math_score'].mean()
Reading_Avg = studentsdf['reading_score'].mean()
Perc_Mathpass = studentsdf['math_score'].sum()/Total_Students
Perc_Readpass = studentsdf['reading_score'].sum()/Total_Students
Perc_Overall = (Perc_Mathpass + Perc_Readpass)/2
students_sub = [Total_Students, Math_Avg, Reading_Avg, Perc_Mathpass, Perc_Readpass, Perc_Overall]

district_df = pd.DataFrame()
district_df = district_df.append({'Total Schools': Total_Schools,
                                  'Total Students': Total_Students,
                                  'Total Budget': Total_Budget,
                                  'Average Math Score': Math_Avg,
                                  'Average Reading Score': Reading_Avg,
                                  '% Passing Math': Perc_Mathpass,
                                  '% Passing Reading': Perc_Readpass,
                                  '% Overall Passing Rate': Perc_Overall
                                 }, ignore_index=True)
#Ordered df
district_df = district_df[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score',
                        '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

# district_df["Total Schools"] = district_df["Total Schools"].map("{:,}").format
# district_df["Total Students"] = district_df["Total Students"].map("{:,}").format
# district_df["Total Budget"] = district_df["Total Budget"].map("${:,}").format
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15.0,39170.0,24649428.0,78.985371,81.87784,78.985371,81.87784,80.431606


School Summary

In [6]:
#Groupby school, get: Total Students, Total Passing Math/Reading (#s not %)
school_group = mergedf.groupby('school_name')
school_sum = pd.DataFrame(school_group.sum())
school_avg = pd.DataFrame(school_group.mean())

#New column calculations
schoolperf = schoolsdf.loc[:,['school_name','school_type','school_budget']]
school_sum = school_sum.loc[:, ['student_tracker','math_pass','reading_pass']]
perc_math_pass = (school_sum['math_pass']/school_sum['student_tracker']) *100
school_sum['perc_math_pass'] = perc_math_pass
perc_read_pass = (school_sum['reading_pass']/school_sum['student_tracker']) *100
school_sum['perc_read_pass'] = perc_read_pass
perc_ovll_pass  = (perc_math_pass + perc_read_pass)/2
school_sum['perc_ovll_pass'] = perc_ovll_pass
school_avg = school_avg.loc[:, ['math_score','reading_score']]

sum_avg_merge = school_sum.join(school_avg, how= 'outer')
sum_avg_merge = sum_avg_merge.reset_index()
schoolperf_merge = pd.merge(schoolperf, sum_avg_merge, on = 'school_name')
perstud_budget = schoolperf_merge['school_budget']/schoolperf_merge['student_tracker']
schoolperf_merge['perstud_budget'] = perstud_budget

schoolperf_merge = schoolperf_merge[['school_name','school_type','student_tracker','school_budget',
                                    'perstud_budget','math_score','reading_score','perc_math_pass',
                                    'perc_read_pass','perc_ovll_pass']]

main_df = schoolperf_merge.rename(index=str, columns= {"school_name":"School Name", "school_type":"School Type",
                                                       "student_tracker":"Total Students", "school_budget":"Total School Budget",
                                                       "perstud_budget":"Per Student Budget", "math_score":"Average Math Score",
                                                       "reading_score":"Average Reading Score", "perc_math_pass":"% Passing Math",
                                                       "perc_read_pass":"% Passing Reading", 
                                                       "perc_ovll_pass":"% Overall Passing Rate"
                                                      })
main_df.set_index('School Name', inplace=True)

main_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391


Top Performing Schools (By Passing Rate)

In [7]:
main_df.sort_values(by=('% Overall Passing Rate'), ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


Bottom Performing Schools (By Passing Rate)

In [8]:
main_df.sort_values(by=('% Overall Passing Rate'), ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


Math Scores By GradeMath Scores By Grade

In [9]:
# grade_avg = pd.DataFrame(school_group.mean())
# grade_avg
grade_group = mergedf.groupby(['school_name','student_grade'])
grade_avg = pd.DataFrame(grade_group.mean())
grade_score = grade_avg[['math_score','reading_score']]
grade_score = grade_score.reset_index(level = 'student_grade')
grade_score = grade_score.reset_index()
#grade_score
math_score = grade_score[['school_name','student_grade','math_score']]
math_score = math_score.pivot(index='school_name', columns='student_grade')['math_score']
math_score[['9th','10th','11th','12th']]

student_grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


Reading Scores By Grade

In [10]:
#Use 'grade_score' from above to display reading scores by grade
read_score = grade_score[['school_name','student_grade','reading_score']]
read_score = read_score.pivot(index='school_name', columns='student_grade')['reading_score']
read_score[['9th','10th','11th','12th']]

student_grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


Scores By School Spending

In [11]:
#Using main_df, calculate scores by spending(per student)
#Create new column 'Spending Ranges (Per Student)' 
#Define a function to populate new column
def f(row):
    if row['Per Student Budget'] < 585:
        val = '<$585'
    elif (row['Per Student Budget'] >= 585 and row['Per Student Budget'] < 615):
        val = '$585-615'
    elif (row['Per Student Budget'] >= 615 and row['Per Student Budget'] < 645):
        val = '$615-645'
    else:
        val = '>$645'
    return val

main_df['Spending Ranges (Per Student)'] = main_df.apply(f, axis=1)
spRng_group = main_df.groupby('Spending Ranges (Per Student)')
spRng_avg = pd.DataFrame(spRng_group.mean())
spRng_avg = spRng_avg.loc[:,['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
spRng_avg = spRng_avg.reset_index()

spRng_avg = spRng_avg.set_index([[1,2,0,3], 'Spending Ranges (Per Student)'])
spRng_avg = spRng_avg.sort_index()
spRng_avg

,,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,Spending Ranges (Per Student),,,,,
0,<$585,83.455399,83.933814,93.460096,96.610877,95.035486
1,$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
2,$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
3,>$645,76.997210,81.027843,66.164813,81.133951,73.649382


Scores By School Size

In [12]:
#Using main_df, calculate scores by school size
#Create new column 'School Size' 
#Define a function to populate new column
def h(row):
    if row['Total Students'] < 1000:
        val = 'Small (<1000)'
    elif (row['Total Students'] >= 1000 and row['Total Students'] < 2000):
        val = 'Medium (1000-2000)'
    elif row['Total Students'] >= 2000:
        val = 'Large (>2000)'
    return val

main_df['School Size'] = main_df.apply(h, axis=1)
scSize_group = main_df.groupby('School Size')
scSize_avg = pd.DataFrame(scSize_group.mean())
scSize_avg = scSize_avg.loc[:,['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
scSize_avg = scSize_avg.reset_index()

scSize_avg = scSize_avg.set_index([[2,1,0], 'School Size'])
scSize_avg = scSize_avg.sort_index()
scSize_avg

,,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,School Size,,,,,
0,Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
1,Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
2,Large (>2000),77.746417,81.344493,69.963361,82.766634,76.364998


Scores By School Type

In [13]:
scType_group = main_df.groupby('School Type')
scType_avg = pd.DataFrame(scType_group.mean())
scType_avg = scType_avg.loc[:,['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]
scType_avg = scType_avg.sort_index()
scType_avg

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
